In [ ]:
import os

In [ ]:
os.listdir()

In [ ]:
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = './train'
test_dir = './validation'

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with validation cat/dog pictures
test_cats_dir = os.path.join(test_dir, 'cats')
test_dogs_dir = os.path.join(test_dir, 'dogs')

In [ ]:
train_cat_fnames = os.listdir( train_cats_dir )
train_dog_fnames = os.listdir( train_dogs_dir )

print(train_cat_fnames[:10])
print(train_dog_fnames[:10])

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range = 40, 
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest'
                                  )
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(150,150),
                                                   batch_size=20,
                                                   class_mode='binary')    

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                target_size=(150,150),
                                                batch_size=20,
                                                class_mode='binary')


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation='elu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3), activation='elu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3), activation='elu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='elu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='nadam', metrics=['acc'])

In [ ]:
history = model.fit(train_generator, steps_per_epoch=100, epochs=100, validation_data=test_generator, validation_steps=50, verbose=2)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title("Training and validation accuracy")
plt.figure()

In [ ]:
#Prediction 
#prediction
import numpy as np
from tensorflow.keras.preprocessing import image
path = './validation/dogs/dog.2000.jpg'
test_image = image.load_img(path,target_size=(150,150))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
result = model.predict(test_image)
if result[0][0] == 1:
    prediction = 'dog'
    print(prediction)
else:
    prediction = 'cat'
    print(prediction)

In [ ]:
#Visualize 
import numpy as np 
import random 
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import matplotlib.pyplot as plt 

# Define a new model that will take an image as input and will output 
# intermediate representations for all layers in the previous model after the first 

successive_outputs  = [layer.output for layer in model.layers[1:]]
visualize_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

#Prepare a random input image from the training set. 
cat_img_files = [os.path.join(train_cats_dir, f) for f in train_cat_fnames]
dog_img_files = [os.path.join(train_dogs_dir, f) for f in train_dog_fnames] 
img_path = random.choice(cat_img_files + dog_img_files)
img = load_img(img_path, target_size = (150,150)) 
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

# scale by 1/255

x /= 255.0 

# run the image through the network, thus obtaining all intermediate representations 
successive_feature_maps = visualize_model.predict(x)

# These are the names of the layers so you can have them as part of of the plot 
layer_names = [layer.name for layer in model.layers]

# Display the representations 
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
    if len(feature_map.shape) == 4:
        n_features = feature_map.shape[-1]
        size = feature_map.shape[1]
        # Tile the images in this matrix 
        display_grid = np.zeros((size, size * n_features))
        
        #Postprocess the feature to be visually palatable
        
        for i in range(n_features):
            x = feature_map[0, :, :, i]
            x -= x.mean()
            x /= x.std()
            x  *= 64
            x += 128
            x = np.clip(x, 0, 255).astype('uint8')
            display_grid[:, i*size:(i+1)*size] = x #Tile each filter into a horizontal grid
            
        scale = 20./n_features
        plt.figure(figsize=(scale*n_features,scale))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='viridis')
            


## Transfer Learning

In [ ]:
# Transfer Learning 

import os 
from tensorflow.keras import layers 
from tensorflow.keras import Model 

from tensorflow.keras.applications.inception_v3 import InceptionV3


# https://www.kaggle.com/datasets/madmaxliu/inceptionv3
local_weights_file = './inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150,150,3),
                                include_top = False, 
                                weights=None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False
    
pre_trained_model.summary()

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7')

last_output = last_layer.output 



from tensorflow.keras.optimizers import RMSprop 

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)
model.compile(optimizer = RMSprop(learning_rate = 0.0001),
             loss='binary_crossentropy',
             metrics=['acc'])


history = model.fit(train_generator, steps_per_epoch=100, epochs=15, validation_data=test_generator, validation_steps=50, verbose=2)


In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title("Training and validation accuracy")
plt.figure()

## Multi-class classification 

In [ ]:
train_dir='./rps_dataset'
test_dir='./rps-cv-images'

train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range = 40, 
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest'
                                  )
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(300,300),
                                                   batch_size=5,
                                                   class_mode='categorical')    

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                target_size=(300,300),
                                                batch_size=5,
                                                class_mode='categorical')


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(3,3), activation = 'relu', input_shape=(300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(loss = 'binary_crossentropy',
             optimizer='adam',
             metrics=['acc'])

history = model.fit(train_generator, steps_per_epoch=10, epochs=15, validation_data=test_generator, validation_steps=50, verbose=2)



In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title("Training and validation accuracy")
plt.figure()

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
path = './rps-cv-images/paper/0a3UtNzl5Ll3sq8K.png'
test_image = image.load_img(path,target_size=(300,300))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
#images = np.vstack([test_image])

result = model.predict(test_image)
print(result)

plt.imshow(image.load_img(path,target_size=(300,300)))
